In [1]:
%reload_ext autoreload
%autoreload 2

import os
import sys
import time
import cv2

from joblib import Parallel, delayed

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from data_manager import *
from metadata import *

import matplotlib.pyplot as plt
%matplotlib inline

from skimage.transform import rotate

Setting environment for Gordon


In [2]:
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.tools as tls
tls.set_credentials_file(username='mistycheney', api_key='cjx2w6a4si')

/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:90: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning


In [3]:
cell_masks_normalized = bp.unpack_ndarray_file('/home/yuncong/csd395/CSHL_cells/fractal_dim/cell_masks_normalized.bp')

In [21]:
n_cells = len(cell_masks_normalized)

In [76]:
from registration_utilities import *

In [112]:
! rm -r /home/yuncong/csd395/CSHL_cells/gallery/random/*

In [114]:
n = 1000

colors = random_colors(n)

indices = np.random.choice(np.arange(n_cells), n)
for r, i in enumerate(indices):
    
    contour_mask = np.zeros_like(cell_masks_normalized[i])
    contour_pts = np.array(find_contour_points(cell_masks_normalized[i].astype(np.int), sample_every=1)[1][0], np.int)
    contour_mask[contour_pts[:,1], contour_pts[:,0]] = 1
    xmin, xmax, ymin, ymax = bbox_2d(contour_mask)      
    m = contour_mask[ymin:ymax+1, xmin:xmax+1].copy()

    m_rgba = np.zeros(m.shape+(4,), np.uint8)
    m_rgba[m > 0, :3] = colors[r]
    m_rgba[m > 0, 3] = 255
    
    contour_output_dir = create_if_not_exists('/home/yuncong/csd395/CSHL_cells/gallery/random/random%d/contours' % (n))
    imsave(contour_output_dir + '/cellMask_random%dContours_%d_%d.png' % (n,r, i), m_rgba)
    
    xmin, xmax, ymin, ymax = bbox_2d(cell_masks_normalized[i])  
    m = cell_masks_normalized[i][ymin:ymax+1, xmin:xmax+1].copy()

    m_rgba = np.zeros(m.shape+(4,), np.uint8)
    m_rgba[m > 0, :3] = colors[r]
    m_rgba[m > 0, 3] = 255
    
    blob_output_dir = create_if_not_exists('/home/yuncong/csd395/CSHL_cells/gallery/random/random%d/blobs' % (n))
    imsave(blob_output_dir + '/cellMask_random%dBlobs_%d_%d.png' % (n, r, i), m_rgba)


E_dict = []

for r, i in enumerate(indices):
    e1, e2, e3, e4, e5 = E[i, :5]
    E_dict.append({'e1':e1, 'e2':e2, 'e3':e3, 'e4':e4, 'e5':e5, 'id':r, 'index':i})

import json
json.dump(E_dict, open('/home/yuncong/csd395/CSHL_cells/gallery/random/random%d/embedding_random%d.json' % (n, n), 'w'))

/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/skimage/io/_io.py:131: UserWarning:

/home/yuncong/csd395/CSHL_cells/gallery/random/random1000/blobs/cellMask_random1000Blobs_66_149557.png is a low contrast image

/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/skimage/io/_io.py:131: UserWarning:

/home/yuncong/csd395/CSHL_cells/gallery/random/random1000/blobs/cellMask_random1000Blobs_186_66605.png is a low contrast image

/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/skimage/io/_io.py:131: UserWarning:

/home/yuncong/csd395/CSHL_cells/gallery/random/random1000/contours/cellMask_random1000Contours_357_189927.png is a low contrast image

/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/skimage/io/_io.py:131: UserWarning:

/home/yuncong/csd395/CSHL_cells/gallery/random/random1000/blobs/cellMask_random1000Blobs_357_189927.p

In [115]:
! cd /home/yuncong/csd395/CSHL_cells/gallery/; tar -czf random.tar.gz random

In [6]:
E = bp.unpack_ndarray_file('/home/yuncong/csd395/CSHL_cells/fractal_dim/embeddingAllEigen_nystromSample100.bp')

In [23]:
E_normalized = bp.unpack_ndarray_file('/home/yuncong/csd395/CSHL_cells/fractal_dim/embeddingAllEigenNormalized_nystromSample100.bp')

In [4]:
from sklearn.cluster import KMeans, MiniBatchKMeans

n_classes = 100

kmeans = MiniBatchKMeans(n_clusters=n_classes)
kmeans.fit(E_original_order);

print np.bincount(kmeans.labels_, minlength=n_classes)

[1954  829 2163 2190 1999 2224 1138 3610 1659 1611 1792 1340 3102 1223 2196
 1642 2632 1084 2840 1291 2143 1114  547 2435 2577  456 2524 1745 3261 3868
 1377  811 2395 1898 2576  276  914  817 1942 3343 2069 2085 2847 2328 2295
 3079 3567 2342 1616 1386 3719 2092 1110 2735 2078 1527 1788 1812 3417 1865
 1261 1521 1833 1067  768 1325 3715 1309 2330 2202 2128 1987  263 1565 1924
 1566 3169 1170  695 1979 2701 1885 2410 2411 1701 1206 3009 1765 3028 2111
 1441 3566 2941 1794 3075  920 1440 2172 1711 1594]


In [5]:
colors = np.asarray(random_colors(n_classes))
colors = ['rgb(%d,%d,%d)'%(r,g,b) for r,g,b in colors]

In [6]:
import plotly

In [26]:
data = []


trace = go.Scattergl(
    x = E[:,0],
    y = E[:,1],
    mode = 'markers',
    name = 'cluster %d'%i,
    marker = dict(
        line = dict(width = 1)
    )
)
data += [trace]

plotly.offline.plot(data, filename='/home/yuncong/csd395/CSHL_cells/fractal_dim/embedding_nystromSample100_2d0and1.html', auto_open=False)

'file:///home/yuncong/csd395/CSHL_cells/fractal_dim/embedding_nystromSample100_2d0and1.html'

In [27]:
data = []


trace = go.Scattergl(
    x = E_normalized[:,1],
    y = E_normalized[:,2],
    mode = 'markers',
    name = 'cluster %d'%i,
    marker = dict(
        line = dict(width = 1)
    )
)
data += [trace]

plotly.offline.plot(data, filename='/home/yuncong/csd395/CSHL_cells/fractal_dim/embeddingNormalized_nystromSample100_2d1and2.html', auto_open=False)

'file:///home/yuncong/csd395/CSHL_cells/fractal_dim/embeddingNormalized_nystromSample100_2d1and2.html'

In [7]:
data = []

for i in range(n_classes):

    m = kmeans.labels_ == i

    trace = go.Scattergl(
        x = E_original_order[m,0],
        y = E_original_order[m,2],
        mode = 'markers',
        name = 'cluster %d'%i,
        marker = dict(
            line = dict(width = 1),
            color = colors[i]
        )
    )
    data += [trace]

plotly.offline.plot(data, filename='/home/yuncong/csd395/CSHL_cells/fractal_dim/embedding_nystromSample100_2d0and2.html', auto_open=False)

IndexError: index 2 is out of bounds for axis 1 with size 2